# Baseline model traning and prediction **(notebook deprecated)**
**NOTE: I initially tried to use the LSTM model, but after more thought, I decided to use the MLP model.**
~~It's baseline model traning and prediction. **This model uses only data from FPL, and it's build with simple RNN.**~~

In [26]:
# import libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [27]:
data_path = os.path.dirname(os.getcwd()) + '\\data\\Fantasy-Premier-League'
merged_gw_2021_path = data_path + '\\2020-21\\gws\\merged_gw.csv'

In [28]:
merged_gw_2021 = pd.read_csv(merged_gw_2021_path, encoding='latin-1')

In [29]:
merged_gw_2021.shape

(24365, 36)

In [30]:
merged_gw_2021.head()

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,Aaron Connolly,FWD,Brighton,0.5,0,0,-3,0,0.3,78,...,1,32.0,1,0,0,0,55,True,0,1
1,Aaron Cresswell,DEF,West Ham,2.1,0,0,11,0,11.2,435,...,0,0.0,1,0,0,0,50,True,0,1
2,Aaron Mooy,MID,Brighton,0.0,0,0,0,0,0.0,60,...,1,0.0,0,0,0,0,50,True,0,1
3,Aaron Ramsdale,GK,Sheffield Utd,2.5,0,0,12,0,0.0,483,...,0,0.0,1,0,0,0,50,True,0,1
4,Abdoulaye DoucourÃ©,MID,Everton,1.3,0,0,20,1,44.6,512,...,0,4.0,3,0,0,0,55,False,0,1


In [31]:
# sort rows by column 'GW' ascending
merged_gw_2021.sort_values(by='GW', inplace=True)

## Data preprocessing

In [32]:
# count number of unique names
unique_names = merged_gw_2021['name'].nunique()
print('Number of unique names: ', unique_names)

Number of unique names:  712


In [33]:
# one hot encode 'element' column from merged_gw_2021
merged_gw_2021 = pd.get_dummies(merged_gw_2021, columns=['element'])

In [34]:
# one hot encode 'position' column from merged_gw_2021
merged_gw_2021 = pd.get_dummies(merged_gw_2021, columns=['position'])

In [35]:
merged_gw_2021.head()

,name,team,xP,assists,bonus,bps,clean_sheets,creativity,fixture,goals_conceded,...,element_708,element_709,element_710,element_711,element_712,element_713,position_DEF,position_FWD,position_GK,position_MID
0,Aaron Connolly,Brighton,0.5,0,0,-3,0,0.3,7,2,...,0,0,0,0,0,0,0,1,0,0
289,Maxime Le Marchand,Fulham,0.6,0,0,0,0,0.0,2,0,...,0,0,0,0,0,0,1,0,0,0
288,Matt Ritchie,Newcastle,0.7,0,0,0,0,0.0,6,0,...,0,0,0,0,0,0,0,0,0,1
287,Matt Phillips,West Brom,0.8,0,0,0,0,0.0,5,0,...,0,0,0,0,0,0,0,0,0,1
286,Matt Macey,Arsenal,2.0,0,0,0,0,0.0,2,0,...,0,0,0,0,0,0,0,0,1,0


In [36]:
teams_2021_path = data_path + '\\2020-21\\teams.csv'
teams_2021 = pd.read_csv(teams_2021_path, encoding='latin-1')

In [37]:
# change 'opponent_team' column from merged_gw_2021 to 'name' column from teams_2021 where 'opponent_team' is equal to 'id'
merged_gw_2021['opponent_team'] = merged_gw_2021['opponent_team'].map(teams_2021.set_index('id')['name'])

In [38]:
# one hot encode 'team' and 'opponent_team' column from merged_gw_2021
merged_gw_2021 = pd.get_dummies(merged_gw_2021, columns=['team', 'opponent_team'])

In [39]:
merged_gw_2021

,name,xP,assists,bonus,bps,clean_sheets,creativity,fixture,goals_conceded,goals_scored,...,opponent_team_Liverpool,opponent_team_Man City,opponent_team_Man Utd,opponent_team_Newcastle,opponent_team_Sheffield Utd,opponent_team_Southampton,opponent_team_Spurs,opponent_team_West Brom,opponent_team_West Ham,opponent_team_Wolves
0,Aaron Connolly,0.5,0,0,-3,0,0.3,7,2,0,...,0,0,0,0,0,0,0,0,0,0
289,Maxime Le Marchand,0.6,0,0,0,0,0.0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
288,Matt Ritchie,0.7,0,0,0,0,0.0,6,0,0,...,0,0,0,0,0,0,0,0,1,0
287,Matt Phillips,0.8,0,0,0,0,0.0,5,0,0,...,0,0,0,0,0,0,0,0,0,0
286,Matt Macey,2.0,0,0,0,0,0.0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23892,Pierre-Emile HÃ¸jbjerg,2.0,0,0,15,0,4.7,373,2,0,...,0,0,0,0,0,0,0,0,0,0
23893,Ãaglar SÃ¶yÃ¼ncÃ¼,0.0,0,0,16,0,0.7,373,4,0,...,0,0,0,0,0,0,1,0,0,0
23894,Harry Lewis,-0.5,0,0,0,0,0.0,377,0,0,...,0,0,0,0,0,0,0,0,1,0
23874,Kenneth Zohore,0.0,0,0,0,0,0.0,372,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
# drop 'fixture', 'kickoff_time', 'round' column from merged_gw_2021
merged_gw_2021.drop(['fixture', 'kickoff_time', 'round'], axis=1, inplace=True)

In [41]:
# change was_home column to binary
merged_gw_2021['was_home'] = merged_gw_2021['was_home'].map({True: 1, False: 0})

In [42]:
merged_gw_2021

,name,xP,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,...,opponent_team_Liverpool,opponent_team_Man City,opponent_team_Man Utd,opponent_team_Newcastle,opponent_team_Sheffield Utd,opponent_team_Southampton,opponent_team_Spurs,opponent_team_West Brom,opponent_team_West Ham,opponent_team_Wolves
0,Aaron Connolly,0.5,0,0,-3,0,0.3,2,0,3.4,...,0,0,0,0,0,0,0,0,0,0
289,Maxime Le Marchand,0.6,0,0,0,0,0.0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
288,Matt Ritchie,0.7,0,0,0,0,0.0,0,0,0.0,...,0,0,0,0,0,0,0,0,1,0
287,Matt Phillips,0.8,0,0,0,0,0.0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
286,Matt Macey,2.0,0,0,0,0,0.0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23892,Pierre-Emile HÃ¸jbjerg,2.0,0,0,15,0,4.7,2,0,1.8,...,0,0,0,0,0,0,0,0,0,0
23893,Ãaglar SÃ¶yÃ¼ncÃ¼,0.0,0,0,16,0,0.7,4,0,4.3,...,0,0,0,0,0,0,1,0,0,0
23894,Harry Lewis,-0.5,0,0,0,0,0.0,0,0,0.0,...,0,0,0,0,0,0,0,0,1,0
23874,Kenneth Zohore,0.0,0,0,0,0,0.0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0


## Split to train and test sets and scale data

In [43]:
# split merged_gw_2021 into test set, where GW is greater than 35, and rest of the data is for training
train = merged_gw_2021[merged_gw_2021['GW'] < 35]
test = merged_gw_2021[merged_gw_2021['GW'] >= 35]

In [44]:
X_train = train.drop(['name', 'total_points'], axis=1).copy()
y_train = train[['total_points']].copy()

print(X_train.shape, y_train.shape)

(21323, 784) (21323, 1)


In [45]:
X_test = test.drop(['name', 'total_points'], axis=1).copy()
y_test = test[['total_points']].copy()

print(X_test.shape, y_test.shape)

(3042, 784) (3042, 1)


In [46]:
X_scaler = MinMaxScaler(feature_range=(0, 1))
scaled_X_train = X_scaler.fit_transform(X_train)
print(scaled_X_train.shape)

y_scaler = MinMaxScaler(feature_range=(0, 1))
scaled_y_train = y_scaler.fit_transform(y_train).reshape(-1)
print(scaled_y_train.shape)

(21323, 784)
(21323,)


## Prepare data with timeseries generator

In [47]:
n_input = unique_names * 3 #how many samples/timesteps to look in the past in order to forecast the next sample
n_features = X_train.shape[1] # how many predictors/Xs/features we have to predict y
b_size = 32 # Number of timeseries samples in each batch
generator = TimeseriesGenerator(scaled_X_train, scaled_y_train, length=n_input, batch_size=b_size)

print(generator[0][0].shape)

(32, 2136, 784)


## Build LSTM model

In [48]:
model = Sequential()
model.add(LSTM(150, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 150)               561000    
                                                                 
 dense_1 (Dense)             (None, 1)                 151       
                                                                 
Total params: 561,151
Trainable params: 561,151
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(generator, epochs=5)

Epoch 1/5
600/600 [==============================] - 2777s 5s/step - loss: 0.0068
Epoch 2/5
525/600 [=========================>....] - ETA: 6:24 - loss: 0.0058